In [1]:
!pip install mediapipe opencv-python

In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [8]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
r_counter = 0 
r_e = 0
l_e = 0 


stage = None
r_stage = None
 # 5 minutes from now
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    import time
    timeout = time.time() + 60*1
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
           # print(landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].visibility)
            left_ear = landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].visibility
            left_ear = float("{:.2f}".format(left_ear))
            right_ear = landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].visibility
            right_ear = float("{:.2f}".format(right_ear))
            
            # Get coordinates
            #Left Hand
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            #Right Hand
            r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            #left Ear
            l_ear = [landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x,landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y]
            
             #right Ear
            r_ear = [landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            angle= float("{:.2f}".format(angle))
            r_angle = calculate_angle(r_shoulder, r_elbow, r_wrist)
            r_angle= float("{:.2f}".format(r_angle))
            #print(r_angle)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            #Right angle 
            cv2.putText(image, str(r_angle), 
                           tuple(np.multiply(r_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            
              #left ear  
            cv2.putText(image,str(left_ear), 
                           tuple(np.multiply(l_ear, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
             #right ear  
            cv2.putText(image,str(right_ear), 
                           tuple(np.multiply(r_ear, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
                                
            
            # Curl counter logic
            if angle > 160:
                stage = "down"
            if angle < 30 and stage =='down':
                stage="up"
                counter +=1
               # print(counter)
                
            if r_angle > 160:
                r_stage = "down"
            if r_angle < 30 and r_stage =='down':
                r_stage="up"
                r_counter +=1
               # print(r_counter)
            
            if right_ear < 0.98:
                r_e +=1
                
                
            if left_ear < 0.98:
                l_e +=1
                
                
            test = 0
            if test == 1 or time.time() > timeout:
                if counter > 0 :
                    print("its has been 1 min  and the left hand is moved ",counter," times , Right hand is moved ",r_counter," times and face turned towards  right side  is ",r_e,"and left side  is ",l_e)
                    data = "its has been 1 min  and the left hand is moved ",counter," times , Right hand is moved ",r_counter," times and face turned towards  right side  is ",r_e,"and left side  is ",l_e
                    print(data)
                else :
                    data = "No Movement Captured"
                    print(data)
                with open("read.txt", "a") as text_file:
                    print(data, file=text_file)
                timeout = time.time() + 60*1  # 5 minutes from now
                test=0
            test = test - 1
            
            
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'LEFT', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
         #  Right Setup status box
        cv2.rectangle(image, (250,0), (450,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'RIGHT', (265,12), 
                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(r_counter), 
                    (250,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (315,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, r_stage, 
                   (290,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
       
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

its has been 1 min  and the left hand is moved  6  times , Right hand is moved  3  times and face turned towards  right side  is  0 and left side  is  0
('its has been 1 min  and the left hand is moved ', 6, ' times , Right hand is moved ', 3, ' times and face turned towards  right side  is ', 0, 'and left side  is ', 0)
its has been 1 min  and the left hand is moved  6  times , Right hand is moved  4  times and face turned towards  right side  is  6 and left side  is  0
('its has been 1 min  and the left hand is moved ', 6, ' times , Right hand is moved ', 4, ' times and face turned towards  right side  is ', 6, 'and left side  is ', 0)
its has been 1 min  and the left hand is moved  6  times , Right hand is moved  5  times and face turned towards  right side  is  6 and left side  is  0
('its has been 1 min  and the left hand is moved ', 6, ' times , Right hand is moved ', 5, ' times and face turned towards  right side  is ', 6, 'and left side  is ', 0)
its has been 1 min  and the lef

KeyboardInterrupt: 

In [ ]:
import time
timeout = time.time() + 60*1  # 5 minutes from now
while True:
    test = 0
    if test == 1 or time.time() > timeout:
       print("its has been 1 min")
       timeout = time.time() + 60*1  # 5 minutes from now
       test=0
    test = test - 1